<a href="https://colab.research.google.com/github/BorisLoveDev/agents-of-uniform-difficulty/blob/main/get_answer_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
from tensorflow.keras.models import load_model
import pandas as pd
from tensorflow.keras.datasets import mnist


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Загрузка тестового набора данных MNIST
(_, _), (x_test_mnist, y_test_mnist) = mnist.load_data()

# Нормализация данных и добавление измерения канала
x_test_mnist = x_test_mnist / 255.0
x_test_mnist = np.expand_dims(x_test_mnist, axis=-1)

In [3]:
# Загрузка скомбинированных данных
x_combined = np.load('/content/drive/MyDrive/all_difficulty_data_mnist/x_combined_difficulty.npy')
y_combined = np.load('/content/drive/MyDrive/all_difficulty_data_mnist/y_combined_difficulty.npy')
y_combined_ohe = np.eye(11)[y_combined]  # Получаем one-hot encoding для 'не знаю' класса


In [4]:
# Загрузка всех 10 моделей
models = []
for i in range(1, 11):
    model_path = f'/content/drive/MyDrive/all_difficulty_data_mnist/weights_100rcnn_normal_train_data/model_{i}_best.keras'
    model = load_model(model_path)
    models.append(model)

In [10]:
def test_model_on_mnist_data(model, x_data, y_data):
    predictions = model.predict(x_data).argmax(axis=1)
    correct = (predictions == y_data).sum()
    total = x_data.shape[0]
    accuracy = correct / total
    return accuracy

# Тестирование каждой модели на MNIST и вывод результатов
for i, model in enumerate(models, 1):
    accuracy = test_model_on_mnist_data(model, x_test_mnist, y_test_mnist)
    print(f"Model {i} accuracy on MNIST test data: {accuracy:.2%}")


313/313 [==============================] - 1s 2ms/step
Model 1 accuracy on MNIST test data: 98.60%
313/313 [==============================] - 1s 3ms/step
Model 2 accuracy on MNIST test data: 95.87%
313/313 [==============================] - 1s 2ms/step
Model 3 accuracy on MNIST test data: 97.26%
313/313 [==============================] - 1s 3ms/step
Model 4 accuracy on MNIST test data: 98.91%
313/313 [==============================] - 1s 3ms/step
Model 5 accuracy on MNIST test data: 97.74%
313/313 [==============================] - 1s 3ms/step
Model 6 accuracy on MNIST test data: 90.49%
313/313 [==============================] - 1s 3ms/step
Model 7 accuracy on MNIST test data: 96.43%
313/313 [==============================] - 1s 2ms/step
Model 8 accuracy on MNIST test data: 93.38%
313/313 [==============================] - 1s 2ms/step
Model 9 accuracy on MNIST test data: 96.70%
313/313 [==============================] - 1s 2ms/step
Model 10 accuracy on MNIST test data: 95.40%


In [11]:
def test_model_on_combined_data(model, x_data, y_data):
    predictions = model.predict(x_data).argmax(axis=1)
    results = np.where(predictions == y_data, 1, np.where(predictions == 10, 0, -1))
    return results

# Тестирование всех моделей и сохранение результатов
combined_results = np.array([test_model_on_combined_data(model, x_combined, y_combined) for model in models])


329/329 [==============================] - 1s 2ms/step


In [12]:
# Преобразование результатов в DataFrame и сохранение в CSV
combined_df = pd.DataFrame(combined_results)
combined_df.to_csv('/content/drive/MyDrive/all_difficulty_data_mnist/combined_test_results.csv', index=False, header=False)

print("Результаты тестирования на скомбинированном наборе данных сохранены.")


Результаты тестирования на скомбинированном наборе данных сохранены.


In [13]:
# Показываем последние строки DataFrame
print(combined_df.tail())

# Сохраняем DataFrame в CSV без индексов и заголовков
combined_df.to_csv('/content/drive/MyDrive/all_difficulty_data_mnist/combined_test_results.csv', index=False, header=False)

print("Результаты тестирования на скомбинированном наборе данных сохранены.")


   0      1      2      3      4      5      6      7      8      9      ...  \
5      1      1      1      1      1      1      1      1      1      1  ...   
6      1      1      1      1      1      1      1      1      1      1  ...   
7      1      1      1      1      1      1      1      1      1      1  ...   
8      1      1      1      1      1      1      1      1      1      1  ...   
9      1      1      1      1      1      1      1      1      1      1  ...   

   10490  10491  10492  10493  10494  10495  10496  10497  10498  10499  
5      0      0      0      0      0      0      0      0      0      0  
6      0      0      0      0      0      0      0      0      0      0  
7      0      0      0      0      0      0      0      0      0      0  
8      0      0      0      0      0      0      0      0      0      0  
9      0      0      0      0      0      0      0      0      0      0  

[5 rows x 10500 columns]
Результаты тестирования на скомбинированном набор